<a href="https://colab.research.google.com/github/tipiorgup/Tutorial_IDP_SAPs/blob/main/SLATM_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install qmllib

In [4]:
import pickle
!pip install mdtraj
import mdtraj as md

with open('mbtypes.pickle', 'rb') as f:
    mbtypes = pickle.load(f)

with open('stored_values.pkl', 'rb') as f:
    stored_values = pickle.load(f)

with open('HHHHH-A_ensemble.pkl', 'rb') as f:
    ensemble = pickle.load(f)

slatm_file='slatm.pkl'

In [5]:
from qmllib import *
from qmllib.representations import generate_slatm

In [6]:
def calculate_slatm(stored_values, ensembles, slatm_file, mbtypes):
    d = {'A':1, 'R':2, 'N':3, 'D':4, 'C':5, 'Q':6, 'G':7, 'E':8, 'H':9, 'I':10,
             'L':11, 'K':12, 'M':13, 'F':14, 'P':15, 'S':16, 'T':17, 'W':18, 'Y':19, 'V':20}
    if stored_values['Simulation type'] == 'Single peptide':
        try:
            error_mol=[]
            slatm={}
            numbers_n={}
            n_residues=len(stored_values['Sequence'])
            res_simple=list(stored_values['Sequence'])
            n_chains=int(1000/n_residues)
            # chains=np.reshape(np.arange(ensembles[stored_values['peptide_pairs']][0][0].n_residues),(n_chains,len(stored_values['Sequence'])))
            beads=res_simple*n_chains

            numbers = {stored_values['Sequence']: [d[ni] for ni in beads]}
            slatm = {stored_values['Sequence']: {}}
            #Check original code and we create a SLATM per snapshot, here we use the last snapshot (4) of the last ensmeble (4), otherwise go into two loops
            slatm[stored_values['peptide_pairs']] = []
            slatm_data = generate_slatm(
                ensembles[stored_values['peptide_pairs']][4][4].xyz,
                numbers[stored_values['Sequence']],
                mbtypes=mbtypes,
                rcut=20,
                dgrids=[1, 0.5],
                sigmas=[0.3, 0.2]
            )
            slatm[stored_values['Sequence']].append(slatm_data)

            with open(slatm_file, 'wb') as handle:
                pickle.dump(slatm, handle, protocol=pickle.HIGHEST_PROTOCOL)

            return slatm
        except Exception as e:
            print(f"Error calculating SLATM: {e}")
            return None
    else:
        try:
              n_residues = len(stored_values['Sequence 1'])
              res_simple0 = list(stored_values['Sequence 1'])
              res_simple1 = list(stored_values['Sequence 2'])
              n_mol = ensembles[stored_values['peptide_pairs']][0][0].n_residues
              n_chains = int(1000/n_residues)

              chains = np.reshape(np.arange(n_mol), (n_chains, n_residues))
              beads0 = res_simple0 * (int(int(stored_values['Processed_Ratio'].split('-')[0])/n_residues))
              beads1 = res_simple1 * (int((1000-int(stored_values['Processed_Ratio'].split('-')[0]))/n_residues))

              numbers = {stored_values['peptide_pairs']: [d[ni] for ni in beads0] + [d[ni] for ni in beads1]}
              slatm = {stored_values['peptide_pairs']: {}}
              slatm[stored_values['peptide_pairs']] = []
              slatm_data = generate_slatm(
                  ensembles[stored_values['peptide_pairs']][4][4].xyz,
                  numbers[stored_values['peptide_pairs']],
                  mbtypes=mbtypes,
                  rcut=5,
                  dgrids=[2, 36],
                  sigmas=[0.3, 0.2]
              )
              slatm[stored_values['peptide_pairs']].append(slatm_data)

              with open(slatm_file, 'wb') as handle:
                  pickle.dump(slatm, handle, protocol=pickle.HIGHEST_PROTOCOL)

              return slatm
        except Exception as e:
              print(f"Error calculating SLATM: {e}")
              return None

In [ ]:
calculate_slatm(stored_values, ensemble, slatm_file, mbtypes)